In [1]:
import pandas as pd
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

In [2]:
Chase_df = pd.read_csv("Data/Chase_Activity.csv")
Chase_df.head()

,Details,Posting Date,Description,Amount,Type,Balance,Check or Slip #
DEBIT,01/05/2024,CAPITAL ONE CRCARDPMT 3WQ2TCJXTBJAFZE WE...,-140.01,ACH_DEBIT,,NaN,NaN
DEBIT,01/04/2024,STATE FARM RO 08 CPC-CLIENT 13 S 1284956913 WE...,-130.02,ACH_DEBIT,,NaN,NaN
DEBIT,01/03/2024,AKTRP529 TRAN000518 000005180015325 WE...,-1200.00,MISC_DEBIT,,NaN,NaN
DEBIT,01/02/2024,Zelle payment to Harshal JPM99a791yxl,-10.90,CHASE_TO_PARTNERFI,,NaN,NaN
DEBIT,01/02/2024,HUNTINGTON BANKS IL PAYMENT 20060879691 WE...,-358.07,ACH_DEBIT,,NaN,NaN


In [3]:
Chase_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162 entries, DEBIT to DEBIT
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Details          162 non-null    object 
 1   Posting Date     162 non-null    object 
 2   Description      162 non-null    float64
 3   Amount           162 non-null    object 
 4   Type             162 non-null    object 
 5   Balance          8 non-null      float64
 6   Check or Slip #  0 non-null      float64
dtypes: float64(3), object(4)
memory usage: 10.1+ KB


In [4]:
Chase_df['Details'] = Chase_df['Details'].str.slice(0, 2) +'/' + Chase_df['Details'].str.slice(6, 10)
Chase_df.head()

,Details,Posting Date,Description,Amount,Type,Balance,Check or Slip #
DEBIT,01/2024,CAPITAL ONE CRCARDPMT 3WQ2TCJXTBJAFZE WE...,-140.01,ACH_DEBIT,,NaN,NaN
DEBIT,01/2024,STATE FARM RO 08 CPC-CLIENT 13 S 1284956913 WE...,-130.02,ACH_DEBIT,,NaN,NaN
DEBIT,01/2024,AKTRP529 TRAN000518 000005180015325 WE...,-1200.00,MISC_DEBIT,,NaN,NaN
DEBIT,01/2024,Zelle payment to Harshal JPM99a791yxl,-10.90,CHASE_TO_PARTNERFI,,NaN,NaN
DEBIT,01/2024,HUNTINGTON BANKS IL PAYMENT 20060879691 WE...,-358.07,ACH_DEBIT,,NaN,NaN


In [7]:
df = pd.DataFrame()
df['ds'] = Chase_df['Details']
df['y'] = abs(Chase_df['Description'])

In [8]:
df_month_year = df.groupby(["ds"])
df_month_year = df_month_year["y"].sum()
df_month_year = df_month_year.reset_index()
df_month_year.head()

,ds,y
0,01/2022,2667.47
1,01/2023,2873.14
2,01/2024,1839.00
3,02/2022,1769.71
4,02/2023,2368.37


In [9]:
df_month_year["ds_new"] = df_month_year['ds'].str.slice(3,7) + '-' + df_month_year['ds'].str.slice(0,2) + '-01'
df_month_year.head()

,ds,y,ds_new
0,01/2022,2667.47,2022-01-01
1,01/2023,2873.14,2023-01-01
2,01/2024,1839.00,2024-01-01
3,02/2022,1769.71,2022-02-01
4,02/2023,2368.37,2023-02-01


In [10]:
df = pd.DataFrame()
df['ds'] = pd.to_datetime(df_month_year['ds_new'])
df['y'] = df_month_year['y']
df.tail()

,ds,y
20,2023-10-01,1066.89
21,2022-11-01,1748.02
22,2023-11-01,883.44
23,2022-12-01,1781.38
24,2023-12-01,1731.26


In [11]:
m = Prophet()
m.fit(df)

11:50:49 - cmdstanpy - INFO - Chain [1] start processing
11:50:49 - cmdstanpy - INFO - Chain [1] done processing


In [12]:
future = m.make_future_dataframe(periods=12 * 1,
                                 freq='M')

In [13]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower',
          'yhat_upper', 'trend',
          'trend_lower', 'trend_upper']].tail(20)

,ds,yhat,yhat_lower,yhat_upper,trend,trend_lower,trend_upper
17,2023-06-01,3731.737533,3260.396854,4218.905912,2997.657726,2997.657726,2997.657726
18,2023-07-01,2092.441598,1626.936495,2556.471970,2900.166348,2900.166348,2900.166348
19,2023-08-01,1927.619260,1461.910786,2390.605431,2799.425257,2799.425257,2799.425257
20,2023-09-01,370.131540,-55.981800,831.201003,2698.684167,2698.684167,2698.684167
21,2023-10-01,1043.908987,589.380007,1498.282815,2601.192788,2601.192788,2601.192788
22,2023-11-01,607.115020,112.932131,1044.879851,2500.451698,2500.451698,2500.451698
23,2023-12-01,1282.633478,845.424505,1746.320459,2402.960319,2402.960319,2402.960319
24,2024-01-01,1790.544676,1344.562357,2264.962371,2302.219229,2302.219229,2302.219229
25,2024-01-31,4235.540140,3787.397115,4699.523681,2204.727850,2204.727807,2204.727899
26,2024-02-29,-2404.413694,-2874.732324,-1940.568545,2110.486185,2110.486046,2110.486343
